<a href="https://colab.research.google.com/github/Camicb/Copper-Forecasting/blob/main/2_Stock_Copper_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Stock copper Forecasting

The Chilean copper stock not only depend of the international situation but also the national productivity. The data provided by COCHILCO have 2 variables:

*   **Fecha** : Datetime
*   **Stock Cobre LME (Toneladas Metricas)**: the stock, in metric tonnes

I will perform the same methods that I used for the price copper forecasting: ARIMAL based model and facebook Prophet. Please refer to [here](https://github.com/Camicb/Copper-Forecasting/blob/main/1_Price_Copper_forecasting.ipynb)  to more details and explanations.

In [ ]:
# Installing required libraries

!pip install pystan -q
!pip install fbprophet -q
!pip install statsmodels -q
!pip install pmdarima -q
!pip install scipy -q

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
from sklearn.metrics import mean_squared_error
from pmdarima.arima import auto_arima

In [ ]:
# For reproducibility
np.random.seed(2021)

In [ ]:
# Loading data
stock = pd.read_excel('/content/drive/MyDrive/DATASETS/Copper/stock_cobre.xlsx')

In [ ]:
# Inspecting data
stock.tail()
print(' ')
stock.info()
print(' ')
stock.describe()

In [ ]:
# Redefining variables for simplification purposes 
stock.columns = ['ds', 'y']
stock.drop(stock.index[0], inplace=True)
stock.head()

In [ ]:
# Visualizing data
stock.plot(x='ds', y='y')

In [ ]:
# Spliting data into training and testing datasets
train_size_stock = int(stock.shape[0]*0.9)
train_stock = stock[0:train_size_stock] # 90% data
test_stock = stock[train_size_stock:]  # 10% data

## 2.1 ARIMA based models

In [ ]:
# Inspecting ARIMA models

train_stock.index = train_stock['ds']
test_stock.index =test_stock['ds']

model_stock = auto_arima(train_stock['y'], start_p=1, start_q=1,max_p=5, max_q=5, max_d=5, m=1, max_D=5, trace=True, error_action='ignore')

In [ ]:
# Model selection
model_stock.summary()

In [ ]:
# Making predictions
stock_predictions = pd.DataFrame(model_stock.predict(n_periods=len(test_stock)))
stock_predictions.index=test_stock.index
stock_predictions.columns = ['predictions']
stock_predictions

In [ ]:
# Visualizing results
plt.plot(train_stock['y'], label='Training')
plt.plot(test_stock['y'], label='Test')
plt.plot(stock_predictions['predictions'], label='Predictions')
plt.legend()

In [ ]:
# Metrics
mean_squared_error(test_stock['y'], stock_predictions['predictions'])

In the same way that with copper prices, the ARIMA based model was unable to predict correctly the values.

##2.2 Facebook Prophet

In [ ]:
# Creating model
m_stock= Prophet(daily_seasonality=True)
m_stock.fit(train_stock)
future_stock = m_stock.make_future_dataframe(periods=807,freq='D') 
future_stock.tail()

In [ ]:
# Making predictions
forecast_stock = m_stock.predict(future_stock)
forecast_stock[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# Plot the forecast
fig1 = m_stock.plot(forecast_stock, uncertainty=True)

In [ ]:
# Plot forecast components
fig2 = m_stock.plot_components(forecast_stock)

In [ ]:
# Plot test data and predictions
ax_stock=forecast_stock.plot(x='ds',y='yhat',legend=True,label='predictions',figsize=(12,8))
test_stock.plot(x='ds',y='y',legend=True,label='True Test Data',ax=ax_stock)

In [ ]:
# Metrics
y_test = test_stock['y']
y_predict = forecast_stock['yhat'][-(len(test_stock)):]
mean_squared_error(y_test, y_predict)

The results are even worse with Facebook Prophet than with the ARIMA model. This problem requires a more sophisticated method, like neural networks or collecting more data along with other variables.

### Conclusion

As the prices, stock of copper prices does not have a particular trend or seasonality. They highly depend on the national and international markets situation, and national productivity. Values of last year are very different to the past ones and therefore try to predict the future ones is hard.

#  Bibliography

*  https://www.analyticsvidhya.com/blog/2018/10/predicting-stock-price-machine-learningnd-deep-learning-techniques-python/

*  https://towardsdatascience.com/time-series-forecasting-using-auto-arima-in-python-bb83e49210cd

*  https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.AutoARIMA.html#pmdarima.arima.AutoARIMA

*  https://facebook.github.io/prophet/docs/quick_start.html#python-api

*  https://sailajakarra.medium.com/facebook-prophet-for-time-series-cf26be1be274
